In [ ]:
import os
from ogb.lsc import MAG240MDataset
import numpy as np
import torch
import os
import time
import glob
import argparse
import os.path as osp
from tqdm import tqdm

from typing import Optional, List, NamedTuple

import torch
from torch import Tensor
import torch.nn.functional as F
from torch.nn import ModuleList, Sequential, Linear, BatchNorm1d, ReLU, Dropout
from torch.optim.lr_scheduler import StepLR

from pytorch_lightning.metrics import Accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import (LightningDataModule, LightningModule, Trainer,
                               seed_everything)

from torch_sparse import SparseTensor
from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.data import NeighborSampler

from ogb.lsc import MAG240MDataset, MAG240MEvaluator
from root import ROOT
from torch_geometric.data import DataLoader

In [1]:
import gc

In [ ]:
def get_2_hop(idx,n):
    dad_idx=[]
    son_idx=[]
    set_i=set(idx)
    for i in range(n):
        if edge_index[0][i] in set_i:
            dad_idx.append(edge_index[1][i])
        elif edge_index[1][i] in set_i:
            son_idx.append(edge_index[0][i])
        if i%100000000==0:
            print(i)
    return [set(dad_idx),set(son_idx)]

In [ ]:
dataset = MAG240MDataset(ROOT)
edge_index = dataset.edge_index('paper', 'cites', 'paper')
m,n=edge_index.shape
train_idx=dataset.get_idx_split('train')
val_idx=dataset.get_idx_split('valid')
test_idx=dataset.get_idx_split('test')
trva_idx=np.concatenate((train_idx,val_idx))

In [ ]:
trva_i=get_2_hop(trva_idx,n)

In [ ]:
dad_i=trva_i[0]
son_i=trva_i[1]
tr_i=set(trva_idx)
print(len(dad_i))
print(len(son_i))
print(len(tr_i))

In [ ]:
train_edge=[x.tolist() for x in edge_index.T if (x[0] in trson_i or x[1] in dad_i)]
train_edge=np.array(train_edge).T
len(train_edge[0])

In [ ]:
torch.save(train_edge,'trva_edge.pt')

In [ ]:
del edge_index
gc.collect()
#train_edge=torch.load('trva_edge.pt')

In [ ]:
train_edge=torch.from_numpy(train_edge)
row_tmp=train_edge[0]
col_tmp=train_edge[1]
path = f'./paper_to_paper_symmetric.pt'
t = time.perf_counter()
print('Converting adjacency matrix...', end=' ', flush=True)
#edge_index = dataset.edge_index('paper', 'cites', 'paper')
#edge_index = torch.from_numpy(edge_index)
adj_t = SparseTensor(
    row=row_tmp.to(torch.long),col=col_tmp.to(torch.long),
    sparse_sizes=(len(row_tmp), len(row_tmp)),
    is_sorted=True)
torch.save(adj_t.to_symmetric(), path)
print(f'Done! [{time.perf_counter() - t:.2f}s]')